In [1]:
# -*- coding: utf-8 -*-
"""
Created on August 8 2021

@author: FB
"""
import matplotlib.pyplot as plt
import numpy as np

from time import sleep, monotonic

import qcodes as qc
from qcodes import Station, load_or_create_experiment, initialise_database, Measurement, load_by_run_spec, load_by_id
from qcodes.dataset.plotting import plot_dataset, plot_by_id
from qcodes.utils.dataset.doNd import do0d,do1d, do2d


In [2]:
qc.logger.start_all_logging()
station = qc.Station()

qc.initialise_or_create_database_at('F:\QCoDeSLocal\SpinQubit\database\IMEC_AL809789_D18_D3SD4_T4.db') 
exp = load_or_create_experiment(
    experiment_name= 'T4',
    sample_name="QBB16_3_6"
)

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\T4ACQ\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\T4ACQ\.qcodes\logs\210915-10124-qcodes.log


In [34]:
STATION = qc.Station(config_file='Config_15Sep2021_FB.yaml',use_monitor='True')

In [75]:
qdac=STATION.load_instrument('qdac_A_buffered')

In [5]:
dmm = STATION.load_instrument('keysight_A')

Connected to: Keysight Technologies 34465A (serial:MY54501434, firmware:A.02.09-02.37-02.09-00.49-02-01) in 0.22s


# Buffered acquisition with lock-in SR844


In [18]:
lockin=STATION.load_instrument('lockin_SR844')

2021-09-15 09:38:34,608 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 109 ¦ c:\users\t4acq\qcodes\qcodes\instrument\base.py:128: QCoDeSDeprecationWarning: Parameter ch1_databuffer on instrument lockin_SR844 does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  warnings.warn(

2021-09-15 09:38:34,609 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 109 ¦ c:\users\t4acq\qcodes\qcodes\instrument\base.py:139: QCoDeSDeprecationWarning: Parameter ch1_databuffer did not correctly register itself on instrument lockin_SR844. Please check that `instrument` argument is passed from <class 'qcodes.instrument_drivers.stanford_research.driver_SR844_v0d1.ChannelBuffer'> all the way to `_BaseParameter`. This will be an error in the future.
  warnings.warn(

2021-09-15 09:38:34,609 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 109 ¦ c:\users\t4acq\qcodes\qcodes\instrument\base.py:128: QCoDeSDeprecationWarning: Paramete

Connected to: Stanford_Research_Systems SR844 (serial:s/n49388, firmware:ver1.006) in 0.21s


In [35]:
dmm_lockin = STATION.load_instrument('keysight_A_2')

Connected to: Keysight Technologies 34465A (serial:MY54501309, firmware:A.02.14-02.40-02.14-00.49-02-01) in 1.28s


In [79]:
import re
# do2d_buffered takes as channel not (e.g.) qdac.ch01.v but qdac.ch01, as I need later (in ramp_voltages_2d_buffered) the channel number (obtained by int(re.findall("\d+", <string name>)[0]))
def do2d_buffered(slow_chan, slow_vstart, slow_vend, slow_steps , fast_chan, fast_vstart, fast_vend, fast_steps ,trigger_channel,step_length, dmm):    
    meas = Measurement()
    v1s = np.linspace(slow_vstart, slow_vend, slow_steps)
    v2s = np.linspace(fast_vstart, fast_vend, fast_steps)
    meas.register_parameter(slow_chan.v)
    meas.register_parameter(fast_chan.v)
    meas.register_custom_parameter('buffer', 'Buffer', 'V', setpoints=(slow_chan.v,fast_chan.v,))
    with meas.run() as datasaver: 
        dmm.init_measurement()   
        meas_time = qdac.ramp_voltages_2d_buffered( slow_chans=[int(re.findall("\d+", slow_chan.short_name)[0])], slow_vstart=[slow_vstart], slow_vend=[slow_vend],
                                  fast_chans=[int(re.findall("\d+", fast_chan.short_name)[0])], fast_vstart=[fast_vstart], fast_vend=[fast_vend], trigger_channel = int(re.findall("\d+", trigger_channel.short_name)[0]),
                                  slow_steps = slow_steps, fast_steps = fast_steps,
                                  step_length=step_length)               
        sleep(meas_time)
        data = dmm.fetch()
        datasaver.add_result((slow_chan.v, np.repeat(v1s,fast_steps)),
                             (fast_chan.v, np.tile(v2s,slow_steps)),
                             ('buffer', data))    
        dataset2D = datasaver.dataset

In [88]:
dmm_lockin.device_clear() #necessary after timeout error
dmm_lockin.reset() #can be commented
slow_steps = 51 #number of pixel along slow axis
fast_steps = 51 #number of pixel along fast axis
all_gates(2)
# qdac.ch03.v(0.4)
# qdac.ch05.v(0.4)
# sleep(1)
#dmm.timeout(5)
dmm_lockin.display.text('buffering...') # Displays the text to speed up commands of the dmm
dmm_lockin.range(10) #range in volts
dmm_lockin.aperture_mode('ON')
dmm_lockin.NPLC(0.02) #{0.2, 1, 0.06, 100, 10, 0.02}
dmm_lockin.trigger.source('EXT') #before was IMM
dmm_lockin.trigger.slope('POS')
dmm_lockin.trigger.count(slow_steps*fast_steps)
dmm_lockin.trigger.delay(0.0)

dmm_lockin.sample.count(1)
dmm_lockin.sample.timer('MIN')
#dmm_lockin.sample.pretrigger_count(0)

do2d_buffered(qdac.ch03, 0.300, 0.600, slow_steps, qdac.ch05, 0.300, 0.600, fast_steps ,qdac.ch22,0.002, dmm_lockin)

dmm_lockin.display.clear() # Returns display to its normal state


Starting experimental run with id: 762. 


# Buffered acquisition with ithaco

In [103]:
import re
# do2d_buffered takes as channel not (e.g.) qdac.ch01.v but qdac.ch01, as I need later (in ramp_voltages_2d_buffered) the channel number (obtained by int(re.findall("\d+", <string name>)[0]))
def do2d_buffered(slow_chan, slow_vstart, slow_vend, slow_steps , fast_chan, fast_vstart, fast_vend, fast_steps ,trigger_channel,step_length, dmm):    
    meas = Measurement()
    v1s = np.linspace(slow_vstart, slow_vend, slow_steps)
    v2s = np.linspace(fast_vstart, fast_vend, fast_steps)
    meas.register_parameter(slow_chan.v)
    meas.register_parameter(fast_chan.v)
    meas.register_custom_parameter('buffer', 'Buffer', 'A', setpoints=(slow_chan.v,fast_chan.v,))
    with meas.run() as datasaver: 
        dmm.init_measurement()   
        meas_time = qdac.ramp_voltages_2d_buffered( slow_chans=[int(re.findall("\d+", slow_chan.short_name)[0])], slow_vstart=[slow_vstart], slow_vend=[slow_vend],
                                  fast_chans=[int(re.findall("\d+", fast_chan.short_name)[0])], fast_vstart=[fast_vstart], fast_vend=[fast_vend], trigger_channel = int(re.findall("\d+", trigger_channel.short_name)[0]),
                                  slow_steps = slow_steps, fast_steps = fast_steps,
                                  step_length=step_length)               
        sleep(meas_time)
        data = dmm.fetch()*1e-8*(-1)
        datasaver.add_result((slow_chan.v, np.repeat(v1s,fast_steps)),
                             (fast_chan.v, np.tile(v2s,slow_steps)),
                             ('buffer', data))    
        dataset2D = datasaver.dataset

In [104]:
dmm.device_clear()  #necessary after timeout error
dmm.reset() #can be commented
slow_steps = 51 #number of pixel along slow axis
fast_steps = 51 #number of pixel along fast axis
all_gates(2)
# qdac.ch03.v(0.4)
# qdac.ch05.v(0.4)
# sleep(1)
#dmm.timeout(5)
dmm.display.text('buffering...') # Displays the text to speed up commands of the dmm
dmm.range(1) #range in volts
dmm.aperture_mode('ON')
dmm.NPLC(0.02) #{0.2, 1, 0.06, 100, 10, 0.02}
dmm.trigger.source('EXT') #before was IMM
dmm.trigger.slope('POS')
dmm.trigger.count(slow_steps*fast_steps)
dmm.trigger.delay(0.0)

dmm.sample.count(1)
dmm.sample.timer('MIN')
#dmm_lockin.sample.pretrigger_count(0)

do2d_buffered(qdac.ch03, 0.400, 0.500, slow_steps, qdac.ch05, 0.400, 0.500, fast_steps ,qdac.ch22,0.002, dmm)

dmm.display.clear() # Returns display to its normal state


Starting experimental run with id: 771. 


2021-09-15 16:29:39,655 ¦ qcodes.dataset.measurements ¦ WARNING ¦ measurements ¦ __exit__ ¦ 635 ¦ An exception occured in measurement with guid: aaaaaaaa-0000-0000-0000-017be9dce7b1;
Traceback:
Traceback (most recent call last):
  File "C:\Users\T4ACQ\AppData\Local\Temp/ipykernel_10124/3373077191.py", line 18, in do2d_buffered
    datasaver.add_result((slow_chan.v, np.repeat(v1s,fast_steps)),
  File "c:\users\t4acq\qcodes\qcodes\dataset\measurements.py", line 214, in add_result
    self._validate_result_shapes(results_dict)
  File "c:\users\t4acq\qcodes\qcodes\dataset\measurements.py", line 416, in _validate_result_shapes
    raise ValueError(f'Incompatible shapes. Parameter '
ValueError: Incompatible shapes. Parameter buffer has shape (1,), but its setpoint qdac_A_buffered_chan03_v has shape (2601,).



ValueError: Incompatible shapes. Parameter buffer has shape (1,), but its setpoint qdac_A_buffered_chan03_v has shape (2601,).